In [9]:
# Import libraries

import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing



In [10]:
# random forest generation returning the mean absolute error

from sklearn.ensemble import RandomForestRegressor

def get_random_forest_mae(X_trn,X_tst,y_trn,y_tst):
  mdl = RandomForestRegressor(random_state=1)
  mdl.fit(X_trn,y_trn)
  y_tst_prd = mdl.predict(X_tst)
  mae = mean_absolute_error(y_tst,y_tst_prd)
  return(mae)

In [11]:

df = pd.read_csv("austin_311.csv")
df.head()

c:\Users\hamza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Service Request (SR) Number,SR Description,Method Received,SR Status,SR Location,Street Number,Street Name,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate,(Latitude.Longitude),Council District,Created Date,Closed Date
0,19-00090956,Animal - Proper Care,Phone,Closed,"4609 RIBBECKE AVE, AUSTIN, TX 78721",4609,RIBBECKE,AUSTIN,78721.0,TRAVIS,30.280581,-97.685318,"(30.28058076, -97.68531793)",1.0,2019-03-14 03:02:15,2020-11-23 01:41:21
1,20-00135805,Tree Issue ROW,Mobile Device,Closed,"3521 WEST AVE, AUSTIN, TX 78705",3521,WEST,AUSTIN,78705.0,TRAVIS,30.302857,-97.742312,"(30.30285681, -97.74231249)",9.0,2020-04-07 07:06:32,2020-11-23 12:02:05
2,20-00052942,Pothole Repair,Web,Closed,"7900 E BEN WHITE BLVD WB, AUSTIN, TX",7900,BEN WHITE BLVD WB,AUSTIN,78741.0,TRAVIS,30.219838,-97.688440,"(30.21983764, -97.68844011)",3.0,2020-02-12 01:55:04,2020-02-12 12:57:35
3,19-00467964,ARR Dead Animal Collection,Phone,Closed,"7100 METROPOLIS DR, AUSTIN, TX 78744",7100,METROPOLIS,AUSTIN,78744.0,TRAVIS,30.206036,-97.702592,"(30.20603574, -97.70259216)",2.0,2019-12-10 02:17:04,2019-12-11 09:57:54
4,19-00468033,Austin Code - Request Code Officer,Phone,Closed,"2401 GROVE BLVD, AUSTIN, TX 78741",2401,GROVE,AUSTIN,78741.0,TRAVIS,30.221664,-97.707177,"(30.22166434, -97.70717698)",3.0,2019-12-10 02:51:24,2019-12-10 03:00:05


In [12]:
# define our variables, we want to determine the status so that is our y, all other factors are treated as independent to that event

y = df.loc[:,["SR Status"]]
x = df.drop(["SR Status","SR Location","Created Date", "Closed Date", "(Latitude.Longitude)", "Street Number", "Service Request (SR) Number"], axis = 1)

# # split the data into train and test data

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=1)

In [13]:
# selecting numeric features of the dataset
num_columns = [c for c in x.columns if x[c].dtype in ['int64', 'float64']]
x_numeric = x[num_columns]

# using x_numeric.isna() I see there are NaN values in the numeric columns, so I will replace with the mean
x_numeric = x_numeric.fillna(x_numeric.mean())

x_numeric_train, x_numeric_test, y_numeric_train, y_numeric_test = train_test_split(x_numeric, y, test_size=0.2, random_state=1)

x_numeric.head()

,Zip Code,Latitude Coordinate,Longitude Coordinate,Council District
0,78721.0,30.280581,-97.685318,1.0
1,78705.0,30.302857,-97.742312,9.0
2,78741.0,30.219838,-97.688440,3.0
3,78744.0,30.206036,-97.702592,2.0
4,78741.0,30.221664,-97.707177,3.0


In [24]:
x_numeric.isna().sum()

Zip Code                0
Latitude Coordinate     0
Longitude Coordinate    0
Council District        0
dtype: int64

In [20]:
# selecting the categorical feature of the dataset

# cat_columns = [col for col in x.columns if x[col].dtype == 'object']

x_replace_category_train = x_train.copy()
x_replace_category_test = x_test.copy()



le = preprocessing.LabelEncoder()

# for col in cat_columns:
#     x_replace_category_train[col] = le.fit_transform(x_replace_category_train[col])
#     x_replace_category_test[col] = le.fit_transform(x_replace_category_test[col])
    

# after checking the nunique of the categorical feature, we choose 144 as the maximum number of unique values

# attempting to do this before tranforming the categorical feature NEW
# x_replace_category_train = x_train.copy().fillna("-")
# x_replace_category_test = x_test.copy().fillna("-")

cat_obj = [c for c in x.columns if x[c].dtype == 'object' and x[col].nunique() < 144]

# ORIGINAL
# x_replace_category_train = x_train.copy().fillna("-")
# x_replace_category_test = x_test.copy().fillna("-")


for col in cat_obj:
    x_replace_category_train[col] = le.fit_transform(x_replace_category_train[col])
    x_replace_category_test[col] = le.fit_transform(x_replace_category_test[col])

# x_replace_category_test()

print(cat_obj)


['SR Description', 'Method Received', 'Street Name', 'City', 'County']


In [23]:
x_replace_category_test.isna().sum()

SR Description              0
Method Received             0
Street Name                 0
City                        0
Zip Code                 2599
County                      0
Latitude Coordinate      1817
Longitude Coordinate     1817
Council District        10751
dtype: int64

In [21]:
x_replace_category_train.head()


,SR Description,Method Received,Street Name,City,Zip Code,County,Latitude Coordinate,Longitude Coordinate,Council District
128245,10,12,12672,2,NaN,6,3.442387,-105.983195,NaN
740698,127,12,5416,2,78704.0,4,30.234623,-97.795950,5.0
469086,29,12,1059,2,78727.0,4,30.422346,-97.715872,7.0
107841,32,9,7133,2,78701.0,4,30.268293,-97.742382,9.0
119276,2,12,8067,2,78753.0,4,30.359475,-97.682469,4.0


In [15]:
y_columns = [c for c in y.columns if y[c].dtype == 'object']
y_train_rep, y_test_rep = y_train.copy(), y_test.copy()
# y_train_rep.isna().sum()
for c in y_columns:
    y_train_rep[c] = le.fit_transform(y_train_rep[c])
    y_test_rep[c] = le.fit_transform(y_test_rep[c])



In [16]:
get_random_forest_mae(x_replace_category_train[cat_obj + num_columns], x_replace_category_test[cat_obj + num_columns], y_train_rep, y_test_rep)

ValueError: could not convert string to float: '-'